In [1]:
import pandas as pd
import numpy as np
from PIL import Image
import os
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.layers import Input,Lambda,Flatten,Dense
from keras.models import Model
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split


In [2]:
data= pd.read_excel('Cases Meta data.xlsx')
df = data[["CaseID", "VIA"]]
df=df.sort_values(by="CaseID")

In [36]:
# Step 1: Create a list of file paths
image_dir = 'D:\coding\PRML\hobby\cervical cancer\Data'  # Replace with your image directory
file_paths = [os.path.join(image_dir, filename) for filename in os.listdir(image_dir)]
print(file_paths[184])

D:\coding\PRML\hobby\cervical cancer\Data\APV1.jpg


In [3]:
df2= pd.read_excel('Cases - Images.xlsx')
df2

,CaseNumber,File,Type
0,1,AFC0.jpg,Before application of acetic acid
1,1,AFC1.jpg,After application of acetic acid
2,2,AJL0.jpg,Before application of acetic acid
3,2,AJL1.jpg,After application of acetic acid
4,3,AGY0.jpg,Before application of acetic acid
...,...,...,...
415,184,AEU1.jpg,After application of acetic acid
416,185,ADF0.jpg,Before application of acetic acid
417,185,ADF1.jpg,After application of acetic acid
418,186,ABK0.jpg,Before application of acetic acid


In [5]:

# Assuming you have a DataFrame 'df' with 'CaseID' column

# Step 1: Create a list of valid image file paths
image_dir = 'D:/coding/PRML/hobby/cervical cancer/Data'  # Replace with your image directory
valid_image_paths = []

# Create a list of indices to drop
indices_to_drop = []

for i, case_id in enumerate(df['CaseID']):
    image_filename = f"{case_id}1.jpg"  # Assuming images are named as 'caseID1.jpg'
    image_path = os.path.join(image_dir, image_filename)
    
    if os.path.exists(image_path):
        valid_image_paths.append(image_path)
    else:
        indices_to_drop.append(i)

# Drop rows with missing images
df = df.drop(index=indices_to_drop)

# Add the "Path" column to the DataFrame
df["Path"] = valid_image_paths

# Display the updated DataFrame
print(df.head())

    CaseID                   VIA  \
86     AAA              Negative   
147    AAB              Positive   
180    AAC  Suspicious of cancer   
96     AAD              Positive   
123    AAE              Positive   

                                                  Path  
86   D:/coding/PRML/hobby/cervical cancer/Data\AAA1...  
147  D:/coding/PRML/hobby/cervical cancer/Data\AAB1...  
180  D:/coding/PRML/hobby/cervical cancer/Data\AAC1...  
96   D:/coding/PRML/hobby/cervical cancer/Data\AAD1...  
123  D:/coding/PRML/hobby/cervical cancer/Data\AAE1...  


In [6]:
df["Image"]=df["Path"].map(lambda x: np.asarray(Image.open(x)))
df

,CaseID,VIA,Path,Image
86,AAA,Negative,D:/coding/PRML/hobby/cervical cancer/Data\AAA1...,"[[[202, 77, 29], [199, 77, 28], [195, 73, 24],..."
147,AAB,Positive,D:/coding/PRML/hobby/cervical cancer/Data\AAB1...,"[[[111, 50, 22], [111, 49, 24], [111, 50, 31],..."
180,AAC,Suspicious of cancer,D:/coding/PRML/hobby/cervical cancer/Data\AAC1...,"[[[103, 80, 62], [112, 89, 71], [118, 93, 73],..."
96,AAD,Positive,D:/coding/PRML/hobby/cervical cancer/Data\AAD1...,"[[[100, 62, 15], [105, 67, 20], [119, 82, 37],..."
123,AAE,Positive,D:/coding/PRML/hobby/cervical cancer/Data\AAE1...,"[[[70, 22, 10], [48, 6, 0], [49, 19, 11], [62,..."
...,...,...,...,...
44,APF,Negative,D:/coding/PRML/hobby/cervical cancer/Data\APB1...,"[[[142, 112, 86], [143, 113, 87], [137, 110, 8..."
49,API,Negative,D:/coding/PRML/hobby/cervical cancer/Data\APF1...,"[[[255, 196, 163], [254, 194, 160], [251, 191,..."
55,APT,Negative,D:/coding/PRML/hobby/cervical cancer/Data\API1...,"[[[169, 92, 50], [171, 98, 53], [167, 103, 55]..."
56,APV,Negative,D:/coding/PRML/hobby/cervical cancer/Data\APT1...,"[[[126, 105, 58], [133, 109, 65], [132, 98, 61..."


In [40]:
df["Image"][1].shape

(72, 96, 3)

In [41]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
le.fit(df["VIA"])
LabelEncoder()
print(list(le.classes_))

['Negative', 'Positive', 'Suspicious of cancer']


In [42]:
df["VIA"]=le.transform(df["VIA"])
print(df.sample(10))


    CaseID  VIA                                               Path  \
116    ABC    1  D:/coding/PRML/hobby/cervical cancer/Data\ABC1...   
175    AET    2  D:/coding/PRML/hobby/cervical cancer/Data\AES1...   
27     AFJ    0  D:/coding/PRML/hobby/cervical cancer/Data\AFI1...   
23     AGM    0  D:/coding/PRML/hobby/cervical cancer/Data\AGB1...   
147    AAB    1  D:/coding/PRML/hobby/cervical cancer/Data\AAB1...   
159    ADX    1  D:/coding/PRML/hobby/cervical cancer/Data\ADW1...   
12     AFH    0  D:/coding/PRML/hobby/cervical cancer/Data\AFE1...   
21     AHR    0  D:/coding/PRML/hobby/cervical cancer/Data\AHQ1...   
142    AAX    1  D:/coding/PRML/hobby/cervical cancer/Data\AAX1...   
86     AAA    0  D:/coding/PRML/hobby/cervical cancer/Data\AAA1...   

                                                 Image  
116  [[[192, 193, 188], [246, 244, 245], [255, 248,...  
175  [[[49, 9, 0], [51, 11, 1], [58, 15, 8], [64, 2...  
27   [[[0, 8, 2], [0, 7, 2], [0, 6, 2], [0, 5, 2], ...  
2

In [43]:
X= np.asarray(df["Image"].tolist())
X=X/255
X.shape

(185, 72, 96, 3)

In [44]:
from keras.utils import to_categorical
Y=df["VIA"]
Y_cat= to_categorical(Y,num_classes=3)
Y.shape

(185,)

In [45]:
X_train,X_test,Y_train,Y_test=train_test_split(X,Y_cat,test_size=0.2,random_state=42)

In [46]:
inputs=(72,96,3)
vgg=VGG16(include_top=False,input_shape= inputs,weights="imagenet")
vgg.output

<KerasTensor: shape=(None, 2, 3, 512) dtype=float32 (created by layer 'block5_pool')>

In [47]:
for layer in vgg.layers:
    layer.trainable=False

In [48]:
x=Flatten()(vgg.output)
prediction=Dense(3,activation="softmax")(x)
model=Model(inputs=vgg.inputs,outputs=prediction)
model.summary()

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 72, 96, 3)]       0         
                                                                 
 block1_conv1 (Conv2D)       (None, 72, 96, 64)        1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 72, 96, 64)        36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 36, 48, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 36, 48, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 36, 48, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 18, 24, 128)       0   

In [49]:
from keras import optimizers

model.compile(loss="categorical_crossentropy",optimizer="Adam",metrics=['acc'])

batch_size=32
epochs=50

In [50]:
history=model.fit(
    X_train,Y_train,
    epochs=epochs,
    batch_size=batch_size,
    validation_data=(X_test,Y_test),
    verbose=2
)

score=model.evaluate(X_test,Y_test)
print("Test accuracy:",score[1])

Epoch 1/50
5/5 - 7s - loss: 1.0201 - acc: 0.4189 - val_loss: 0.9752 - val_acc: 0.5405 - 7s/epoch - 1s/step
Epoch 2/50
5/5 - 5s - loss: 0.8911 - acc: 0.5878 - val_loss: 0.9960 - val_acc: 0.5676 - 5s/epoch - 1s/step
Epoch 3/50
5/5 - 5s - loss: 0.8314 - acc: 0.6014 - val_loss: 0.9917 - val_acc: 0.5135 - 5s/epoch - 964ms/step
Epoch 4/50
5/5 - 5s - loss: 0.7815 - acc: 0.6419 - val_loss: 0.9879 - val_acc: 0.4865 - 5s/epoch - 923ms/step
Epoch 5/50
5/5 - 5s - loss: 0.7456 - acc: 0.6216 - val_loss: 0.9738 - val_acc: 0.5405 - 5s/epoch - 932ms/step
Epoch 6/50
5/5 - 5s - loss: 0.6943 - acc: 0.6959 - val_loss: 0.9615 - val_acc: 0.5405 - 5s/epoch - 953ms/step
Epoch 7/50
5/5 - 5s - loss: 0.6633 - acc: 0.7365 - val_loss: 0.9539 - val_acc: 0.5405 - 5s/epoch - 949ms/step
Epoch 8/50
5/5 - 5s - loss: 0.6296 - acc: 0.7635 - val_loss: 0.9467 - val_acc: 0.5135 - 5s/epoch - 925ms/step
Epoch 9/50
5/5 - 5s - loss: 0.5987 - acc: 0.7635 - val_loss: 0.9426 - val_acc: 0.5135 - 5s/epoch - 948ms/step
Epoch 10/50
5/5 